In [ ]:
from coursework_functions import (create_guassian_filter, print_2d_array, perform_convolution, test_text)
import numpy as np
import cv2
import os
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
# Create the path to the image
IMG_PATH = os.path.join(os.path.abspath('../'), 'data', 'cat.bmp')

In [ ]:
img = cv2.imread(IMG_PATH) # Read the image file
#img = cv2.bitwise_not(img) # Reads it inverted for some reason, so re-invert it
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to grayscale as color not implemented yet

In [ ]:
plt.subplot(1, 1, 1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.yticks([])

In [ ]:
def point_multiply(image, template):
    sum = 0
    for row in range(len(image)):
        for column in range(len(image[0])):
            sum += image[row][column] * template[row][column]
    return sum
def perform_numpy_convolution(image: np.ndarray, template: np.ndarray):
    # Flip the template around both axes
    inverted_template = np.flip(template)

    x, y = len(image), len(image[0])
    n, m = len(inverted_template), len(inverted_template[0])
    new_x, new_y = x - n + 1, y - m + 1

    sections = []
    for row_offset in tqdm(range(0, new_x)):
        for column_offset in range(0, new_y):
            section = np.zeros(shape=(n, m))
            for row in range(0, n):
                for column in range(0, m):
                    section[row][column] = image[row + row_offset][column + column_offset]
            sections.append(section)

    vfunc = np.vectorize(point_multiply, signature='(x,y),(x,y)->()')
    out = vfunc(sections, inverted_template.tolist())

In [ ]:
# Create the guassian kernel
guassian_filter_matrix = create_guassian_filter(sigma=3)
# Apply kernel to the image
low_pass_image = perform_numpy_convolution(img, guassian_filter_matrix)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.yticks([])

plt.subplot(1, 2, 2)
plt.imshow(low_pass_image, cmap='gray')
plt.title('Low Pass Image')
plt.yticks([])

plt.show()

In [ ]:
# Check dimensions of `img` and `low_pass_image` are the same (zero-padding successful)
print('Original Image Dimensions: '.ljust(30, ' ') + f'{len(img)}x{len(img[0])}')
print('Low Pass Image Dimensions: '.ljust(30, ' ') + f'{len(low_pass_image)}x{len(low_pass_image[0])}')
print(test_text('Equal Dimensions', ((len(img) == len(low_pass_image)) and (len(img[0]) == len(low_pass_image[0])))))